In [1]:
import pandas as pd
pd.set_option('display.max_columns', 45)

In [2]:
weekly = pd.read_parquet('data/weekly.parquet')
weekly = weekly.loc[weekly.index != 3821]
weekly['headshot_url'] = weekly['headshot_url'].fillna('no_url')

In [3]:
depth = pd.read_csv('data/depth.csv', index_col=0).sort_values(by='depth_team')
off = depth.loc[depth.formation=='Offense'].copy().drop(
    ['game_id', 'pfr_game_id', 'player', 'pfr_player_id', 'opponent',
       'defense_snaps', 'defense_pct',
       'st_snaps', 'st_pct'],axis=1)
off

C:\Users\Sharedw\AppData\Local\Temp\ipykernel_30688\3572632482.py:1: DtypeWarning: Columns (17,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  depth = pd.read_csv('data/depth.csv', index_col=0).sort_values(by='depth_team')


,season,team_name,week,game_type,depth_team,last_name,first_name,football_name,formation,gsis_id,jersey_number,position,elias_id,depth_position,full_name,dense_depth,offense_snaps,offense_pct
0,2010,ARI,1.0,REG,1,Brown,Levi,Levi,Offense,00-0025392,67.0,T,BRO548586,LT,Levi Brown,0.0,NaN,NaN
297833,2018,CAR,8.0,REG,1,McCaffrey,Christian,Christian,Offense,00-0033280,23.0,RB,MCC035988,RB,Christian McCaffrey,1.0,64.0,0.98
297832,2018,CAR,8.0,REG,1,Olsen,Gregory,Greg,Offense,00-0025418,88.0,TE,OLS094822,TE,Greg Olsen,1.0,63.0,0.97
297831,2018,CAR,8.0,REG,1,Moton,Taylor,Taylor,Offense,00-0033284,72.0,T,MOT466221,RT,Taylor Moton,1.0,63.0,0.97
297829,2018,CAR,8.0,REG,1,Clark,Christopher,Chris,Offense,00-0025900,77.0,T,CLA232975,LT,Chris Clark,0.0,65.0,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33615,2010,SEA,18.0,REG,3,Fanaika,Pauliasi,Paul,Offense,00-0026943,74.0,G,FAN016656,RG,Paul Fanaika,1.0,NaN,NaN
33616,2010,SEA,18.0,REG,3,Giacomini,Breno,Breno,Offense,00-0026290,68.0,T,GIA095058,RT,Breno Giacomini,7.0,NaN,NaN
33617,2010,SEA,18.0,REG,3,Losman,Jonathan,J.P.,Offense,00-0022912,6.0,QB,LOS588329,QB,J.P. Losman,2.0,NaN,NaN
33618,2010,SEA,18.0,REG,3,Washington,Leon,Leon,Offense,00-0024332,29.0,RB,WAS503628,RB,Leon Washington,2.0,NaN,NaN


In [4]:
depth.loc[(depth.elias_id == 'BRO494712')
          & (depth.week==1)]#.groupby(['season','week']).count()

,season,team_name,week,game_type,depth_team,last_name,first_name,football_name,formation,gsis_id,jersey_number,position,elias_id,depth_position,full_name,dense_depth,game_id,pfr_game_id,player,pfr_player_id,opponent,offense_snaps,offense_pct,defense_snaps,defense_pct,st_snaps,st_pct
295041,2018,BAL,1.0,REG,1,Brown,John,John,Offense,00-0031051,16.0,WR,BRO494712,WR,John Brown,2.0,2018_01_BUF_BAL,201809090rav,John Brown,BrowJo02,BUF,48.0,0.60,0.0,0.0,0.0,0.00
332737,2019,BUF,1.0,REG,1,Brown,John,John,Offense,00-0031051,16.0,WR,BRO494712,WR,John Brown,1.0,2019_01_BUF_NYJ,201909080nyj,John Brown,BrowJo02,NYJ,56.0,0.85,0.0,0.0,0.0,0.00
368822,2020,BUF,1.0,REG,1,Brown,John,John,Offense,00-0031051,16.0,WR,BRO494712,WR,John Brown,2.0,2020_01_NYJ_BUF,202009130buf,John Brown,BrowJo02,NYJ,81.0,0.93,0.0,0.0,0.0,0.00
182943,2015,ARI,1.0,REG,2,Brown,John,John,Offense,00-0031051,16.0,WR,BRO494712,WR,John Brown,3.0,2015_01_NO_ARI,201509130crd,John Brown,BrowJo02,NO,60.0,0.97,0.0,0.0,0.0,0.00
182944,2015,ARI,1.0,REG,2,Brown,John,John,Offense,00-0031051,16.0,WR,BRO494712,WR,John Brown,3.0,2015_01_NO_ARI,201509130crd,Jaron Brown,BrowJa03,NO,13.0,0.21,0.0,0.0,20.0,0.65
150413,2014,ARI,1.0,REG,2,Brown,John,John,Offense,00-0031051,16.0,WR,BRO494712,WR,John Brown,2.0,2014_01_SD_ARI,201409080crd,John Brown,BrowJo02,SD,39.0,0.56,0.0,0.0,0.0,0.00
150414,2014,ARI,1.0,REG,2,Brown,John,John,Offense,00-0031051,16.0,WR,BRO494712,WR,John Brown,2.0,2014_01_SD_ARI,201409080crd,Jaron Brown,BrowJa03,SD,5.0,0.07,0.0,0.0,13.0,0.54
150431,2014,ARI,1.0,REG,2,Brown,John,John,Special Teams,00-0031051,16.0,WR,BRO494712,KR,John Brown,2.0,2014_01_SD_ARI,201409080crd,John Brown,BrowJo02,SD,39.0,0.56,0.0,0.0,0.0,0.00
150432,2014,ARI,1.0,REG,2,Brown,John,John,Special Teams,00-0031051,16.0,WR,BRO494712,KR,John Brown,2.0,2014_01_SD_ARI,201409080crd,Jaron Brown,BrowJa03,SD,5.0,0.07,0.0,0.0,13.0,0.54
256515,2017,ARI,1.0,REG,2,Brown,John,John,Offense,00-0031051,16.0,WR,BRO494712,WR,John Brown,2.0,2017_01_ARI_DET,201709100det,John Brown,BrowJo02,DET,63.0,0.84,0.0,0.0,0.0,0.00


In [5]:
#off['offense_snaps'] = off['offense_snaps'].fillna(0)
#off['offense_pct'] = off['offense_pct'].fillna(0)
off

,season,team_name,week,game_type,depth_team,last_name,first_name,football_name,formation,gsis_id,jersey_number,position,elias_id,depth_position,full_name,dense_depth,offense_snaps,offense_pct
0,2010,ARI,1.0,REG,1,Brown,Levi,Levi,Offense,00-0025392,67.0,T,BRO548586,LT,Levi Brown,0.0,NaN,NaN
297833,2018,CAR,8.0,REG,1,McCaffrey,Christian,Christian,Offense,00-0033280,23.0,RB,MCC035988,RB,Christian McCaffrey,1.0,64.0,0.98
297832,2018,CAR,8.0,REG,1,Olsen,Gregory,Greg,Offense,00-0025418,88.0,TE,OLS094822,TE,Greg Olsen,1.0,63.0,0.97
297831,2018,CAR,8.0,REG,1,Moton,Taylor,Taylor,Offense,00-0033284,72.0,T,MOT466221,RT,Taylor Moton,1.0,63.0,0.97
297829,2018,CAR,8.0,REG,1,Clark,Christopher,Chris,Offense,00-0025900,77.0,T,CLA232975,LT,Chris Clark,0.0,65.0,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33615,2010,SEA,18.0,REG,3,Fanaika,Pauliasi,Paul,Offense,00-0026943,74.0,G,FAN016656,RG,Paul Fanaika,1.0,NaN,NaN
33616,2010,SEA,18.0,REG,3,Giacomini,Breno,Breno,Offense,00-0026290,68.0,T,GIA095058,RT,Breno Giacomini,7.0,NaN,NaN
33617,2010,SEA,18.0,REG,3,Losman,Jonathan,J.P.,Offense,00-0022912,6.0,QB,LOS588329,QB,J.P. Losman,2.0,NaN,NaN
33618,2010,SEA,18.0,REG,3,Washington,Leon,Leon,Offense,00-0024332,29.0,RB,WAS503628,RB,Leon Washington,2.0,NaN,NaN


In [6]:
context_cols = ['player_id', 'player_display_name','position',
       'position_group'
]
game_context = ['week','season_type', 'season', 'opponent_team', 'recent_team', ]
stat_cols = ['completions', 'attempts',
       'passing_yards', 'passing_tds', 'interceptions', 'sacks', 'sack_yards',
       'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards',
       'passing_yards_after_catch', 'passing_first_downs', 'passing_epa',
       'passing_2pt_conversions', 'pacr', 'dakota', 'carries', 'rushing_yards',
       'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost',
       'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions',
       'receptions', 'targets', 'receiving_yards', 'receiving_tds',
       'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards',
       'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa',
       'receiving_2pt_conversions', 'racr', 'target_share', 'air_yards_share',
       'wopr', 'special_teams_tds', 'fantasy_points', 'fantasy_points_ppr',
       'offense_snaps','offense_pct']
rush_stat_cols = [
     'carries', 'rushing_yards',
       'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost',
       'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions',
       'receptions', 'targets', 'receiving_yards', 'receiving_tds',
       'fantasy_points', 'fantasy_points_ppr',
       'offense_snaps','offense_pct'
]

depth_cols = ['season', 'last_name',
       'first_name', 'football_name', 'formation', 'gsis_id', 'jersey_number',
       'position', 'elias_id', 'depth_position', 'full_name']


In [7]:

qb_stats_eng = [
    "completions",
    "attempts",
    "passing_yards",
    "passing_tds",
    "interceptions",
    "sacks",
    "sack_yards",
    "passing_air_yards",
    "passing_yards_after_catch",
    "passing_first_downs",
    "passing_2pt_conversions",
    "fantasy_points_ppr",
       'offense_snaps','offense_pct',
       "completion_pct"
]

rush_stats_eng = [
    "carries",
    "rushing_yards",
    "rushing_tds",
    "rushing_first_downs",
    "receptions",
    "targets",
    "receiving_yards",
    "receiving_tds",
    "fantasy_points",
    "fantasy_points_ppr",
       'offense_snaps','offense_pct',
       'yards_per_carry',
]
receiver_stats_eng = [
    "receptions",
    "targets",
    "receiving_yards",
    "receiving_tds",
    "receiving_air_yards",
    "receiving_yards_after_catch",
    "receiving_first_downs",
    "target_share",
    "air_yards_share",
    "fantasy_points_ppr",
      'offense_snaps','offense_pct'
]
stats_to_eng = list(set(qb_stats_eng + rush_stats_eng + receiver_stats_eng))

In [8]:
temp = off.merge(
    weekly,
    how="outer",
    left_on=["gsis_id", "team_name", "season", "week", "position"],
    right_on=["player_id", "recent_team", "season", "week", "position"],
).sort_values(by=['season','week']).reset_index(drop=True)
temp[context_cols] = temp.groupby(depth_cols)[context_cols].bfill()
temp[context_cols] = temp.groupby(depth_cols)[context_cols].ffill()


In [9]:
temp[['carries_sum', 'rush_yards_sum', 'completions_sum', 'attempts_sum']] = temp.groupby(context_cols)[['carries','rushing_yards', 'completions','attempts']].transform(lambda x: x.rolling(8, min_periods=0).sum() )
temp['yards_per_carry'] = temp['rush_yards_sum'] / temp['carries_sum']
temp['completion_pct'] = temp['completions_sum'] / temp['attempts_sum']
temp= temp.drop(['carries_sum', 'rush_yards_sum', 'completions_sum', 'attempts_sum'], axis=1)

In [10]:
temp.loc[temp.offense_snaps > 0, stat_cols] = temp.loc[temp.offense_snaps > 0, stat_cols].fillna(0)
player_fantasy = temp.copy(deep=True).sort_values(by=['season','week']) 
roll_columns = [x + '_roll' for x in stats_to_eng]
player_fantasy[roll_columns] = player_fantasy.groupby(context_cols)[stats_to_eng].transform(lambda x: x.ewm(span=6, min_periods=0).mean())
#player_fantasy[roll_columns] = player_fantasy.groupby(context_cols)[roll_columns].shift(1)

In [11]:
player_fantasy =player_fantasy.groupby(['team_name','season','week', 'game_type', 'position', 'dense_depth', 'gsis_id']).head(1)

In [12]:
player_fantasy[[x for x in player_fantasy.columns if x not in stat_cols]].drop(
    ["recent_team", "opponent_team"], axis=1
).to_parquet("data/agg/player_weekly_agg.parquet")


In [13]:
agg = player_fantasy.copy()
agg = agg.sort_values(by=['team_name','season','week','full_name'])


In [14]:
def stack_players(df, num_players, index_cols, value_cols):
    'df should be seperated by position already'
    df = df.loc[df.dense_depth < num_players]
    piv = df.pivot(index=index_cols, columns=['position','dense_depth'], values=value_cols).reset_index()
    piv.columns = [x[0] + '_' + x[1] +'_' +  (str(int(x[2]))) if x[2] != '' else x[0] + x[1] for x in piv.columns ]
    return piv

In [15]:
cols_to_agg =  (['gsis_id','full_name'] + roll_columns)

In [16]:
qb_roll_stat_cols = ['gsis_id','full_name'] + [x + '_roll' for x in qb_stats_eng] 
receiver_roll_stat_cols = ['gsis_id','full_name'] + [x + '_roll' for x in receiver_stats_eng] 
rush_stat_cols = ['gsis_id','full_name'] + [x + '_roll' for x in rush_stats_eng]

team_qb_stats = stack_players(agg.loc[(agg.position.isin(['QB'])) & (~agg.week.isna())],2,  ['team_name','season','week', 'game_type'], qb_roll_stat_cols)
wr_temp = stack_players(agg.loc[(agg.position.isin(['WR'])) & (~agg.week.isna())],5,  ['team_name','season','week', 'game_type'], receiver_roll_stat_cols)
te_temp = stack_players(agg.loc[(agg.position.isin(['TE'])) & (~agg.week.isna())],3,  ['team_name','season','week', 'game_type'], receiver_roll_stat_cols)
rb_temp =stack_players(agg.loc[(agg.position.isin(['RB'])) & (~agg.week.isna())],2,  ['team_name','season','week', 'game_type'], receiver_roll_stat_cols)
team_receiver_stats = wr_temp.merge(te_temp, on=['team_name','season','week', 'game_type'])
team_receiver_stats = team_receiver_stats.merge(rb_temp, on=['team_name','season','week', 'game_type'])
team_rb_stats = stack_players(agg.loc[(agg.position.isin(['RB'])) & (~agg.week.isna())],3,  ['team_name','season','week', 'game_type'], rush_stat_cols)

In [17]:
stack_players(agg.loc[(agg.position.isin(['WR'])) & (~agg.week.isna())],5,  ['team_name','season','week', 'game_type'], receiver_roll_stat_cols)

,team_name,season,week,game_type,gsis_id_WR_4,gsis_id_WR_3,gsis_id_WR_0,gsis_id_WR_2,gsis_id_WR_1,full_name_WR_4,full_name_WR_3,full_name_WR_0,full_name_WR_2,full_name_WR_1,receptions_roll_WR_4,receptions_roll_WR_3,receptions_roll_WR_0,receptions_roll_WR_2,receptions_roll_WR_1,targets_roll_WR_4,targets_roll_WR_3,targets_roll_WR_0,...,target_share_roll_WR_2,target_share_roll_WR_1,air_yards_share_roll_WR_4,air_yards_share_roll_WR_3,air_yards_share_roll_WR_0,air_yards_share_roll_WR_2,air_yards_share_roll_WR_1,fantasy_points_ppr_roll_WR_4,fantasy_points_ppr_roll_WR_3,fantasy_points_ppr_roll_WR_0,fantasy_points_ppr_roll_WR_2,fantasy_points_ppr_roll_WR_1,offense_snaps_roll_WR_4,offense_snaps_roll_WR_3,offense_snaps_roll_WR_0,offense_snaps_roll_WR_2,offense_snaps_roll_WR_1,offense_pct_roll_WR_4,offense_pct_roll_WR_3,offense_pct_roll_WR_0,offense_pct_roll_WR_2,offense_pct_roll_WR_1
0,ARI,2010,1.0,REG,00-0027691,00-0026221,00-0022921,00-0027349,00-0025529,Andre Roberts,Early Doucet,Larry Fitzgerald,Stephen Williams,Steve Breaston,NaN,2.0,3.0,0.0,7.0,NaN,3.0,15.0,...,0.05,0.175,NaN,0.09466,0.548544,0.06068,0.174757,NaN,5.7,13.3,0.0,20.200001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ARI,2010,2.0,REG,00-0027691,00-0026221,00-0022921,00-0027349,00-0025529,Andre Roberts,Early Doucet,Larry Fitzgerald,Stephen Williams,Steve Breaston,NaN,2.0,5.333333,1.166667,4.666667,NaN,3.0,13.25,...,0.130208,0.164062,NaN,0.09466,0.443294,0.180839,0.128613,NaN,5.7,14.466667,2.683333,10.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ARI,2010,3.0,REG,00-0027352,00-0027691,00-0022921,00-0027349,00-0025529,Max Komar,Andre Roberts,Larry Fitzgerald,Stephen Williams,Steve Breaston,0.908257,NaN,3.834862,1.091743,4.366972,1.458716,NaN,10.440367,...,0.107638,0.234163,0.079674,NaN,0.458927,0.129119,0.230501,1.429358,NaN,12.728441,2.421101,12.43578,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ARI,2010,4.0,REG,00-0027691,00-0027349,00-0025529,00-0026221,00-0022921,Andre Roberts,Stephen Williams,Steve Breaston,Early Doucet,Larry Fitzgerald,NaN,1.828829,4.366972,2.0,5.057432,NaN,5.105856,6.807339,...,0.075,0.361307,NaN,0.211344,0.230501,0.09466,0.418776,NaN,3.880743,12.43578,5.7,12.678829,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ARI,2010,6.0,REG,00-0027352,00-0026221,00-0022921,00-0027349,00-0025529,Max Komar,Early Doucet,Larry Fitzgerald,Stephen Williams,Steve Breaston,0.943694,2.0,5.057432,1.828829,4.366972,2.054054,3.0,10.656532,...,0.180509,0.234163,0.092375,0.09466,0.418776,0.211344,0.230501,1.881532,5.7,12.678829,3.880743,12.43578,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8934,WAS,2024,9.0,REG,00-0033591,00-0036626,00-0035659,00-0035208,00-0039355,Noah Brown,Dyami Brown,Terry McLaurin,Olamide Zaccheaus,Luke McCaffrey,3.37286,1.095166,4.330347,2.124614,0.794838,4.967869,1.455165,6.084746,...,0.111036,0.03422,0.306995,0.057686,0.385637,0.117016,0.017887,9.825023,3.400525,16.542201,4.801537,1.476593,41.029553,24.232568,49.018919,24.919358,26.551849,0.610212,0.364986,0.745885,0.376746,0.400769
8935,WAS,2024,10.0,REG,00-0036626,00-0033591,00-0035659,00-0035208,00-0039355,Dyami Brown,Noah Brown,Terry McLaurin,Olamide Zaccheaus,Luke McCaffrey,1.356759,3.25689,4.523948,1.800007,0.855555,1.901787,5.599917,6.060246,...,0.106047,0.051838,0.033564,0.298278,0.35604,0.101823,0.096424,2.619785,8.728645,16.472179,3.935177,2.016223,20.117835,46.024288,51.326302,22.326405,26.388532,0.305723,0.697715,0.78755,0.339899,0.403501
8936,WAS,2024,11.0,REG,00-0033591,00-0036626,00-0035208,00-0039355,00-0035659,Noah Brown,Dyami Brown,Olamide Zaccheaus,Luke McCaffrey,Terry McLaurin,2.572266,1.253967,1.569767,0.604922,3.508596,5.114584,1.930085,2.629043,...,0.045807,0.169064,0.324878,0.044113,0.105338,0.081025,0.288839,6.505511,2.268329,3.378239,1.425573,12.302322,47.539473,21.52453,23.96758,24.517022,50.079768,0.719727,

In [18]:
team_qb_stats.to_csv('data/team_qb_stats.csv')
team_receiver_stats.to_csv('data/team_receiver_stats.csv')
team_rb_stats.to_csv('data/team_rb_stats.csv')

In [29]:
qb_rush_stats =stack_players(agg.loc[(agg.position.isin(['QB'])) & (~agg.week.isna())],2,  ['team_name','season','week', 'game_type'], rush_stat_cols)
qb_rush_stats = qb_rush_stats.fillna(0)
te_rush_stats =stack_players(agg.loc[(agg.position.isin(['TE'])) & (~agg.week.isna())],1,  ['team_name','season','week', 'game_type'], rush_stat_cols)
te_rush_stats = te_rush_stats.fillna(0)
wr_rush_stats =stack_players(agg.loc[(agg.position.isin(['WR'])) & (~agg.week.isna())],1,  ['team_name','season','week', 'game_type'], rush_stat_cols)
wr_rush_stats = wr_rush_stats.fillna(0)

C:\Users\Sharedw\AppData\Local\Temp\ipykernel_30688\4220847580.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  qb_rush_stats = qb_rush_stats.fillna(0)
C:\Users\Sharedw\AppData\Local\Temp\ipykernel_30688\4220847580.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  te_rush_stats = te_rush_stats.fillna(0)
C:\Users\Sharedw\AppData\Local\Temp\ipykernel_30688\4220847580.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in

In [30]:
temp = team_rb_stats.merge( qb_rush_stats,  on=['team_name','season','week', 'game_type'])
temp = temp.merge( te_rush_stats,  on=['team_name','season','week', 'game_type'])
temp = temp.merge(wr_rush_stats,  on=['team_name','season','week', 'game_type'])
temp.to_csv('data/team_rushers.csv')

In [31]:
temp

,team_name,season,week,game_type,gsis_id_RB_1,gsis_id_RB_2,gsis_id_RB_0,full_name_RB_1,full_name_RB_2,full_name_RB_0,carries_roll_RB_1,carries_roll_RB_2,carries_roll_RB_0,rushing_yards_roll_RB_1,rushing_yards_roll_RB_2,rushing_yards_roll_RB_0,rushing_tds_roll_RB_1,rushing_tds_roll_RB_2,rushing_tds_roll_RB_0,rushing_first_downs_roll_RB_1,rushing_first_downs_roll_RB_2,rushing_first_downs_roll_RB_0,...,receiving_yards_roll_TE_0,receiving_tds_roll_TE_0,fantasy_points_roll_TE_0,fantasy_points_ppr_roll_TE_0,offense_snaps_roll_TE_0,offense_pct_roll_TE_0,yards_per_carry_roll_TE_0,gsis_id_WR_0,full_name_WR_0,carries_roll_WR_0,rushing_yards_roll_WR_0,rushing_tds_roll_WR_0,rushing_first_downs_roll_WR_0,receptions_roll_WR_0,targets_roll_WR_0,receiving_yards_roll_WR_0,receiving_tds_roll_WR_0,fantasy_points_roll_WR_0,fantasy_points_ppr_roll_WR_0,offense_snaps_roll_WR_0,offense_pct_roll_WR_0,yards_per_carry_roll_WR_0
0,ARI,2010,1.0,REG,00-0027007,00-0026956,00-0026289,Chris Wells,LaRod Stephens-Howling,Tim Hightower,NaN,7.0,13.0,NaN,49.0,54.0,NaN,0.0,1.0,NaN,3.0,3.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,00-0022921,Larry Fitzgerald,0.000000e+00,0.000000,0.0,0.000000e+00,3.000000,15.000000,43.000000,1.000000,10.300000,13.300000,0.000000,0.000000,0.000000
1,ARI,2010,2.0,REG,00-0027007,00-0026956,00-0026289,Chris Wells,LaRod Stephens-Howling,Tim Hightower,NaN,2.916667,11.833333,NaN,20.416667,89.583333,NaN,0.0,1.0,NaN,1.25,2.416667,...,11.000000,0.000000,1.100000,2.100000,0.000000,0.000000,0.0,00-0022921,Larry Fitzgerald,0.000000e+00,0.000000,0.0,0.000000e+00,5.333333,13.250000,66.333333,0.416667,9.133334,14.466667,0.000000,0.000000,0.000000
2,ARI,2010,3.0,REG,00-0022288,00-0026956,00-0026289,Jason Wright,LaRod Stephens-Howling,Tim Hightower,0.833333,1.605505,11.458716,1.25,11.238532,67.293578,0.0,0.0,0.550459,0.0,0.688073,2.678899,...,21.500000,0.000000,2.150000,3.733333,0.000000,0.000000,0.0,00-0022921,Larry Fitzgerald,0.000000e+00,0.000000,0.0,0.000000e+00,3.834862,10.440367,48.201835,0.678899,8.893578,12.728441,0.000000,0.000000,0.000000
3,ARI,2010,4.0,REG,00-0027007,00-0022288,00-0026289,Chris Wells,Jason Wright,Tim Hightower,5.0,0.833333,9.736486,19.0,1.25,50.570946,0.0,0.0,0.337838,1.0,0.0,2.416667,...,17.229358,0.000000,1.722936,3.493578,0.000000,0.000000,0.0,00-0025529,Steve Breaston,4.495413e-01,1.798165,0.0,0.000000e+00,4.366972,6.807339,51.917431,0.449541,8.068807,12.435780,0.000000,0.000000,4.000000
4,ARI,2010,6.0,REG,00-0027007,00-0022288,00-0026289,Chris Wells,Jason Wright,Tim Hightower,5.0,0.833333,9.736486,19.0,1.25,50.570946,0.0,0.0,0.337838,1.0,0.0,2.416667,...,17.229358,0.000000,1.722936,3.493578,0.000000,0.000000,0.0,00-0022921,Larry Fitzgerald,0.000000e+00,0.000000,0.0,0.000000e+00,5.057432,10.656532,51.213964,0.416667,7.621397,12.678829,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8926,WAS,2024,9.0,REG,00-0033699,00-0039686,00-0037746,Austin Ekeler,Colson Yankoff,Brian Robinson,7.823393,NaN,13.97716,38.565892,NaN,62.980699,0.351638,NaN,0.693673,1.783787,NaN,3.693959,...,36.986236,0.153863,4.730119,8.268527,41.644830,0.623787,0.0,00-0035659,Terry McLaurin,1.814953e-01,0.151388,0.0,1.208098e-05,4.330347,6.084746,69.686861,0.896543,12.211853,16.542201,49.018919,0.745885,1.096511
8927,WAS,2024,10.0,REG,00-0033699,00-0033955,00-0037746,Austin Ekeler,Jeremy McNichols,Brian Robinson,9.377111,4.731147,13.97716,40.196897,16.799605,62.980699,0.846381,0.426598,0.693673,3.349393,1.186096,3.693959,...,35.256794,0.109412,4.259172,7.930936,43.769779,0.666029,0.0,00-0035659,Terry McLaurin,1.290238e-01,0.107621,0.0,8.588290e-06,4.523948,6.060246,82.208976,0.637347,11.948230,16.472179,51.326302,0.787550,1.064725
8928,WAS,2024,11.0,REG,00-0033699,00-0033955,00-0037746,Austin Ekeler,Jeremy McNichols,Brian Robinson,7.194421,4.226508,14.966429,30.37482,13.359